In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
scales = [2]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [10]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    filter = (x>50)
    filtered_x = s.where(filter, x)
    return s.variance(filtered_x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [11]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 2, param_scale: 2, scale_input_multiplier: 10) ------------->

+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.057937622 | -0.115875244 | 0         | -0.115875244 | 0.057937622    | 0.115875244      | 0.115875244   | 0             | 0.006713536        | -0.0007207595      | 0.0007207595           |
+--------------+--------------+-----------+--------------+----------------+------------------+---------

scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-280,304],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":5411,"total_const_size":1205,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709726428882}


In [12]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.499417781829834 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 80.5
==== Generating Proof ====
proof:  {'instances': [['31220e5513f6e9b7d0f59d8f019cd197642a431b2136f0311abf6046c9006227', '0100000000000000000000000000000000000000000000000000000000000000', '4201000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x054ced15c315add18b81985f2666dcb5d698d3a2b8cfc49becd7357ee6456029201d1a76e4b42b06fe7e5336b0a58a9d4776d2ab031e217d1861a44ec467a38c224639376adb3faae8713682484d4bcd14161402d78d7b39a63d56f171b8151c155aa158d34381c842c649148aebbf264017fa50d80a614aa9eeb6429bd4790003878783966950b14363f79eb6274263bf542a8581e25325efe51d498f2cc6372eaf7f7890c5bb6dd386be7abd7fbced1bc29d44655a18b977d551b060101305174b0a262abd8f0556bd709f184727a28eb79db981c8b44f986ccaaa8d18cb7a0c39c268e75c447d0f696253973a4ff587080df55e9dc091440dac841295ac110178c9624fe94396932630f6542417ee647e704e6282bc69ea46e88d08ba4f24239e61555f1ecd0d

In [14]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [80.5]
